In [1]:
import cameo
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
from cobra.io import save_json_model
from cameo.flux_analysis.simulation import pfba
import Functions_Modules.gapAnalysis
import cobra.test
import os
from Functions_Modules.curation_tools import *

cobra/io/sbml3.py:23 UserWarning: Install lxml for faster SBML I/O


In [2]:
relative_directory = os.getcwd()
filename = relative_directory + '/Reconstructions/MethylococcusModel2.xml'
model = cameo.load_model(filename)

Add Periplasm as a compartment for the respiration to work

In [3]:
model.compartments

{'bm': 'biomass', 'c': 'cytosol', 'e': 'extracellular'}

In [4]:
model.compartments.update({'p':'periplasm','im':'inner membrane','om':'outer membrane'})

# Methane Oxidation

## sMMO
________________

In [5]:
rxn = model.reactions.MNXR6057_c
rxn.name = 'soluble Methane Monooxygenase'
rxn.subsystem = 'rn00680:Methane metabolism,rn01100:Metabolic pathways,rn01120:Microbial metabolism in diverse environments,rn01200:Carbon metabolism'
rxn.notes['SUBSYSTEM'] = ['rn00680:Methane metabolism,rn01100:Metabolic pathways,rn01120:Microbial metabolism in diverse environments,rn01200:Carbon metabolism']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
# Confidence Score
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Cytosol']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.14.13.25']})
# Gene-Reaction-Rule update based on KEGG (using the old locus tags for now)
rxn.notes.update({'GENE_ASSOCIATION': ['( MCA1194 and MCA1195 and MCA1198 and MCA1196 and MCA1200 and MCA1202 and MCA1205 )']})
rxn.gene_reaction_rule = '( MCA1194 and MCA1195 and MCA1198 and MCA1196 and MCA1200 and MCA1202 and MCA1205 )'
# Substrate and cofactor usage from BRENDA
rxn.notes.update({'COFACTOR':['dinuclear FeIV cluster, NADH']})
# NADH - As suggested by 10.1146/annurev.biochem.76.061505.175355
# Inhibitor from BRENDA
rxn.notes.update({'INHIBITOR':['Cu2+']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'sMMO_c'
model.repair()

{}
MNXR6057_c: ch4_c + nadh_c + h_c + o2_c --> meoh_c + h2o_c + nad_c


## pMMO
_______

In [6]:
# Fix Ubiquinone which wasn't in the correct compartment
model.metabolites.q8_c.notes['SMILES'] = ['COC1=C(OC)C(=O)C(C\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CCC=C(C)C)=C(C)C1=O']
model.metabolites.q8_c.notes['INCHI'] = ['InChI=1S/C49H74O4/c1-36(2)20-13-21-37(3)22-14-23-38(4)24-15-25-39(5)26-16-27-40(6)28-17-29-41(7)30-18-31-42(8)32-19-33-43(9)34-35-45-44(10)46(50)48(52-11)49(53-12)47(45)51/h20,22,24,26,28,30,32,34H,13-19,21,23,25,27,29,31,33,35H2,1-12H3/b37-22+,38-24+,39-26+,40-28+,41-30+,42-32+,43-34+']
print model.metabolites.q8_c.notes
model.metabolites.q8_c.compartment = 'im'
model.metabolites.q8_c.id = 'q8_im'
model.repair()

{'SMILES': ['COC1=C(OC)C(=O)C(C\\C=C(/C)CC\\C=C(/C)CC\\C=C(/C)CC\\C=C(/C)CC\\C=C(/C)CC\\C=C(/C)CC\\C=C(/C)CCC=C(C)C)=C(C)C1=O'], 'KEGG': ['C17569'], 'METACYC': ['UBIQUINONE-8'], 'BRENDA': ['BG18990'], 'CHEBI': ['61683'], 'MXNREF': ['MNXM232'], 'SEED': ['cpd15560'], 'INCHI': ['InChI=1S/C49H74O4/c1-36(2)20-13-21-37(3)22-14-23-38(4)24-15-25-39(5)26-16-27-40(6)28-17-29-41(7)30-18-31-42(8)32-19-33-43(9)34-35-45-44(10)46(50)48(52-11)49(53-12)47(45)51/h20,22,24,26,28,30,32,34H,13-19,21,23,25,27,29,31,33,35H2,1-12H3/b37-22+,38-24+,39-26+,40-28+,41-30+,42-32+,43-34+'], 'LIPIDMAPS': ['LMPR02010005'], 'BIGG': ['q8']}


In [7]:
# Add Ubiquinol-8
q8h2_im = Metabolite('q8h2_im',
                       formula='C49H76O4',
                       name='Ubiquinol-8',
                       compartment='im')
q8h2_im.charge = 0
q8h2_im.notes['MXNREF'] = ['MNXM191']
q8h2_im.notes['BIGG'] = ['q8h2']
q8h2_im.notes['KEGG'] = ['C00390']
q8h2_im.notes['SMILES'] = ['COC1=C(O)C(C)=C(C\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CC\C=C(/C)CCC=C(C)C)C(O)=C1OC']
q8h2_im.notes['INCHI']  = ['InChI=1S/C49H76O4/c1-36(2)20-13-21-37(3)22-14-23-38(4)24-15-25-39(5)26-16-27-40(6)28-17-29-41(7)30-18-31-42(8)32-19-33-43(9)34-35-45-44(10)46(50)48(52-11)49(53-12)47(45)51/h20,22,24,26,28,30,32,34,50-51H,13-19,21,23,25,27,29,31,33,35H2,1-12H3/b37-22+,38-24+,39-26+,40-28+,41-30+,42-32+,43-34+']

In [8]:
# Add Methane-Metabolite in the Periplasm
ch4_p = model.metabolites.ch4_c.copy()
ch4_p.compartment = 'p'
ch4_p.id = 'ch4_p'

In [9]:
# Add Methanol-Metabolite in the Periplasm
meoh_p = model.metabolites.meoh_c.copy()
meoh_p.compartment = 'p'
meoh_p.id = 'meoh_p'

In [10]:
# Add Water-Metabolite in the Periplasm
h2o_p = model.metabolites.h2o_c.copy()
h2o_p.compartment = 'p'
h2o_p.id = 'h2o_p'

In [11]:
# Add Oxygen-Metabolite in the Periplasm
o2_p = model.metabolites.o2_c.copy()
o2_p.compartment = 'p'
o2_p.id = 'o2_p'

In [12]:
# Add Proton-Metabolite in the Periplasm
h_p = model.metabolites.h_c.copy()
h_p.compartment = 'p'
h_p.id = 'h_p'

In [13]:
# Add the reaction itself
rxn = Reaction('pMMO_im')
rxn.name = 'particulate Methane Monooxygenase'
rxn.subsystem = 'rn00680:Methane metabolism,rn01100:Metabolic pathways,rn01120:Microbial metabolism in diverse environments,rn01200:Carbon metabolism'
rxn.notes['SUBSYSTEM'] = ['rn00680:Methane metabolism,rn01100:Metabolic pathways,rn01120:Microbial metabolism in diverse environments,rn01200:Carbon metabolism']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({o2_p: -1.0,
                          ch4_p: -1.0,
                          q8h2_im: -1.0,
                          h2o_p: 1.0,
                          meoh_p: 1.0,
                          model.metabolites.q8_im: 1.0})

In [14]:
# Confidence Score
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.14.18.3']})
# Gene-Reaction-Rule updated
rxn.notes.update({'GENE_ASSOCIATION': ['(((MCA1796 and MCA1797 and MCA1798) or (MCA2853 and MCA2854 and MCA2855)) and MCA0295 )' ]})
rxn.gene_reaction_rule = '(((MCA1796 and MCA1797 and MCA1798) or (MCA2853 and MCA2854 and MCA2855)) and MCA0295)'
# Substrate and cofactor usage from 'Respiration in Methanotrophs' DiSpirito, Zahn 2005
rxn.notes.update({'COFACTOR':['2 Cu2+, 1-2 Fe']})
# NADH - As suggested by 10.1146/annurev.biochem.76.061505.175355
# Inhibitor from BRENDA
rxn.notes.update({'INHIBITOR':['']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'pMMO_im'
model.add_reaction(rxn)

{}
pMMO_im


In [15]:
# Add demand reaction for q8h2
rxn = model.reactions.MNXM90307_out
rxn.name = 'Ubiquinol-8 (out)'
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.build_reaction_from_string('q8h2_im -> ')
rxn.id = 'q8h2_out'
model.repair()

model

## NDH-2
_____

In [16]:
rxn = Reaction('NADH5_im')
rxn.name = 'type-2 nadh:quinone oxidoreductase (non-electrogenic)'
rxn.subsystem = 'aerobic electron transport chain'
rxn.notes['SUBSYSTEM'] =['aerobic electron transport chain']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({q8h2_im: 1.0,
                          model.metabolites.nad_c: 1.0,
                          model.metabolites.nadh_c: -1.0,
                          model.metabolites.h_c: -1.0,
                          model.metabolites.q8_im: -1.0})

In [17]:
# Confidence Score
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.6.5.9']})
# Gene-Reaction-Rule updated
rxn.notes.update({'GENE_ASSOCIATION': ['( MCA1918 )' ]})
rxn.gene_reaction_rule = '( MCA1918 )'
# Substrate and cofactor usage from BRENDA
# Uses only NADH
rxn.notes.update({'COFACTOR':['FAD']})
# Inhibitor info from Cook & Shiemke et al. 2004 doi: 10.1128/JB.186.4.928-937.2004
rxn.notes.update({'INHIBITOR':['Diphenyliodonium']})
rxn.notes.update({'BIGG':['NADH5']})
rxn.notes.update({'MXNREF':['MNXR83430']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'NADH5_im'
model.add_reaction(rxn)

{}
NADH5_im


## MDH
-----------

In [18]:
# Cytochrome cL - Cytochrome c555 - MCA2618
ficytcc555_p = model.metabolites.ficytC_c.copy()
ficytcc555_p.compartment = 'p'
ficytcc555_p.id = 'ficytcc555_p'
ficytcc555_p.formula = 'C42H54FeN8O6S2'
ficytcc555_p.notes['MXNREF'] = ['MNXM5749']
ficytcc555_p.notes['BIGG'] = ['ficytc']
ficytcc555_p.notes['KEGG'] = ['C00923']
ficytcc555_p.notes['CHEBI'] = ['5021','14235','15719']
ficytcc555_p.notes['METACYC'] = ['an oxidized c-type cytochrome']
ficytcc555_p.charge = 3

In [19]:
# Cytochrome cL - Cytochrome c555 - MCA2618
focytcc555_p = model.metabolites.focytC_c.copy()
focytcc555_p.compartment = 'p'
focytcc555_p.id = 'focytcc555_p'
focytcc555_p.formula = 'C42H54FeN8O6S2'
focytcc555_p.notes['MXNREF'] = ['MNXM5750']
focytcc555_p.notes['BIGG'] = ['focytc']
focytcc555_p.notes['KEGG'] = ['C00924']
focytcc555_p.notes['CHEBI'] = ['5044','14248','15983']
focytcc555_p.notes['METACYC'] = ['an oxidized c-type cytochrome']
focytcc555_p.charge = 2

In [20]:
# Periplasminc Formaldehyde 
fald_p = model.metabolites.fald_c.copy()
fald_p.compartment = 'p'
fald_p.id = 'fald_p'
fald_p.formula = 'CH2O'
fald_p.notes['MXNREF'] = ['MNXM56']
fald_p.notes['BIGG'] = ['fald']
fald_p.notes['KEGG'] = ['C00067']
fald_p.notes['CHEBI'] = ['16842', '5142', '24077', '337763', '14274']
fald_p.notes['METACYC'] = ['FORMALDEHYDE']
fald_p.charge = 0

In [21]:
rxn = Reaction('MEOHDH_cyt_p')
rxn.name = 'methanol dehydrogenase (cytochrome c)'
rxn.subsystem = 'rn00680:Methane metabolism,rn01120:Microbial metabolism in diverse environments,rn01100:Metabolic pathways,rn01200:Carbon metabolism'
rxn.notes['SUBSYSTEM'] =['rn00680:Methane metabolism,rn01120:Microbial metabolism in diverse environments,rn01100:Metabolic pathways,rn01200:Carbon metabolism']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({meoh_p: -1.0,
                          fald_p: 1.0,
                          ficytcc555_p: -2.0,
                          focytcc555_p: 2.0,
                          h_p: 2.0})

In [22]:
# Confidence Score: Isolated by Amy Rosenzweig 2014 Structure and Protein–Protein Interactions of Methanol Dehydrogenase from Methylococcus capsulatus
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Periplasm']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.1.2.7']})
# Gene-Reaction-Rule updated                  F           J             C            K           L           moxR          A            F            J         C            K          L          moxR            A             I           G             D          D
rxn.notes.update({'GENE_ASSOCIATION': ['(((( MCA0299 and MCA0300 ) and (MCA1528 and MCA1529 and MCA1530 and MCA1525 and MCA1527)) or (MCA0779 and MCA0780 and MCA0786 and MCA0787 and MCA0788 and MCA0783 and MCA0785)) and MCA0782 and MCA0781 and (MCA0789 or MCA2381))' ]})
rxn.gene_reaction_rule = '(((( MCA0299 and MCA0300 ) and (MCA1528 and MCA1529 and MCA1530 and MCA1525 and MCA1527)) or (MCA0779 and MCA0780 and MCA0786 and MCA0787 and MCA0788 and MCA0783 and MCA0785)) and MCA0782 and MCA0781 and (MCA0789 or MCA2381))'
# MCA0781 is cytochrome cL and thus the specific electron acceptor for MDH

# From 'Membrane-associated quinoprotein formaldehyde dehydrogenase from Methylococcus capsulatus Bath' Zahn et al. 2001
# Homotetramer with a cofactor:subunit ratio of 1:1
rxn.notes.update({'COFACTOR':['1 PQQ, 1 Ca2+']})
# Inhibitor info from same source as above.
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['MEOHDH_cyt']})
rxn.notes.update({'MXNREF':['MNXR84835']})
rxn.notes.update({'KEGG':['R01146']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'MEOHDH_cyt_p'
model.add_reaction(rxn)

{}
MEOHDH_cyt_p


## Methanol Diffusion RXN
-----

In [23]:
rxn = Reaction('MeOHt_im')
rxn.name = 'Diffusion CH4 between Periplasm and Cytosol'
rxn.subsystem = 'Diffusion'
rxn.notes['SUBSYSTEM'] =['Diffusion']
rxn.lower_bound = -1000.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({meoh_p: -1.0,
                          model.metabolites.meoh_c: 1.0})

In [24]:
# Confidence Score:
rxn.notes.update({'CONFIDENCE SCORE':['1']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'MEOHt_im'
model.add_reaction(rxn)

{}
MeOHt_im


## Methane Diffusion RXN
-----

In [25]:
# Check on http://www.membranetransport.org/

rxn = Reaction('CH4t_im')
rxn.name = 'Diffusion CH4 between Periplasm and Cytosol'
rxn.subsystem = 'Diffusion'
rxn.notes['SUBSYSTEM'] =['Diffusion']
rxn.lower_bound = -1000.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({ch4_p: -1.0,
                          model.metabolites.ch4_c: 1.0})

In [26]:
# Confidence Score:
rxn.notes.update({'CONFIDENCE SCORE':['1']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'CH4t_im'
model.add_reaction(rxn)

{}
CH4t_im


## Methane Uptake RXN
-----

In [27]:
rxn = Reaction('R_ch4_in')
rxn.name = 'CH4 (in)'
rxn.subsystem = 'Uptake'
rxn.notes['SUBSYSTEM'] =['Uptake']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({ch4_p: 1.0})

In [28]:
# Confidence Score: Isolated by Amy Rosenzweig 2014 Structure and Protein–Protein Interactions of Methanol Dehydrogenase from Methylococcus capsulatus
rxn.notes.update({'CONFIDENCE SCORE':['1']})
# Localization
rxn.notes.update({'LOCALIZATION':['Outer Membrane']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'ch4_in'
model.add_reaction(rxn)

{'H': 4.0, 'C': 1.0}
R_ch4_in


## Formaldehyde Diffusion RXN
-----

In [29]:
# Check on http://www.membranetransport.org/

rxn = Reaction('FALDt_im')
rxn.name = 'Diffusion Formaldehyde between Periplasm and Cytosol'
rxn.subsystem = 'Diffusion'
rxn.notes['SUBSYSTEM'] =['Diffusion']
rxn.lower_bound = -1000.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({fald_p: -1.0,
                          model.metabolites.fald_c: 1.0})

In [30]:
# Confidence Score: Isolated by Amy Rosenzweig 2014 Structure and Protein–Protein Interactions of Methanol Dehydrogenase from Methylococcus capsulatus
rxn.notes.update({'CONFIDENCE SCORE':['1']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'FALDt_im'
model.add_reaction(rxn)

{}
FALDt_im


## N-FalDH
------------------------

In [31]:
rxn = Reaction('ALDD1_c')
rxn.name = 'NAD+-linked formaldehyde dehydrogenase'
rxn.subsystem = 'rn00625:Chloroalkane and chloroalkene degradation,rn00680:Methane metabolism ,rn01100:Metabolic pathways ,rn01120:Microbial metabolism in diverse environments ,rn01200:Carbon metabolism'
rxn.notes['SUBSYSTEM'] =['rn00625:Chloroalkane and chloroalkene degradation,rn00680:Methane metabolism ,rn01100:Metabolic pathways ,rn01120:Microbial metabolism in diverse environments ,rn01200:Carbon metabolism']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({model.metabolites.for_c: 1.0,
                          model.metabolites.nadh_c: 1.0,
                          model.metabolites.nad_c: -1.0,
                          model.metabolites.h2o_c: -1.0,
                          model.metabolites.fald_c: -1.0,
                          model.metabolites.h_c: 2.0})

In [32]:
# Confidence Score: There are ambiguities about its presence. A previous paper that claims to have 
# purified a soluble NAD+-linked FalDH turned out to have purified MDH and a H4MPT-DH instead. 
# It is more likely that the H4MPT-dependent oxidation of formate is the predominant pathway.
# However, the presence of a true NAD+-linked FaldDH cannot be excluded, as there is further evidence.
# Other papers report that the expression is generally low and its function is primarily detoxification of formaldehyde.
rxn.notes.update({'CONFIDENCE SCORE':['2']})
# Localization
rxn.notes.update({'LOCALIZATION':['Cytosol']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.2.1.46']})
# Gene-Reaction-Rule updated
rxn.notes.update({'GENE_ASSOCIATION': ['' ]})
rxn.gene_reaction_rule = ''
rxn.notes.update({'COFACTOR':['NAD']})
# Inhibitor info from same source as above.
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['ALDD1']})
rxn.notes.update({'MXNREF':['MNXR82728']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'ALDD1_c'
model.add_reaction(rxn)

{}
ALDD1_c


## DL-FalDH
------------------------

In [33]:
rxn = Reaction('FALDDH_im')
rxn.name = 'dye-linked formaldehyde dehydrogenase'
rxn.subsystem = 'aerobic electron transport chain'
rxn.notes['SUBSYSTEM'] =['aerobic electron transport chain']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({q8h2_im: 1.0,
                          model.metabolites.for_c: 1.0,
                          model.metabolites.fald_c: -1.0,
                          model.metabolites.h2o_c: -1.0,
                          model.metabolites.q8_im: -1.0,
                          model.metabolites.h_c: 1.0})

In [34]:
# Confidence Score: Isolated by Zahn et al. 2001
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.2.2.B1']})
# Gene-Reaction-Rule updated
rxn.notes.update({'GENE_ASSOCIATION': ['(MCA2155)' ]})
rxn.gene_reaction_rule = '(MCA2155)'
# From 'Membrane-associated quinoprotein formaldehyde dehydrogenase from Methylococcus capsulatus Bath' Zahn et al. 2001
# Homotetramer with a cofactor:subunit ratio of 1:1
rxn.notes.update({'COFACTOR':['PQQ']})
# Inhibitor info from same source as above.
rxn.notes.update({'INHIBITOR':['phenylhydrazine, Zn2+, Fe2+, Cu2+']})
# The identifier for a (ubi)quinone(-8) reducing DL-FalDH does not exist. There is a controversy whether there is a direct
# coupling or whether the enzyme actually uses Q8 as a cofactor. Until this is resolved I am going with what the
# majority of papers suggest: Q8-pool usage instead of direct coupling.
rxn.notes.update({'BIGG':['FALDDH']})
rxn.notes.update({'MXNREF':['MNXR82728']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'FALDDH_im'
model.add_reaction(rxn)

{}
FALDDH_im


## FDH
------------------------

In [35]:
model.reactions.FDH_c.name = 'formate:NADP+ oxidoreductase'
model.reactions.FDH_c.subsystem = 'rn00680:Methane metabolism,rn01120:Microbial metabolism in diverse environments,rn00630:Glyoxylate and dicarboxylate metabolism,rn01100:Metabolic pathways,rn01200:Carbon metabolism'
model.reactions.FDH_c.notes['SUBSYSTEM'] =['rn00680:Methane metabolism,rn01120:Microbial metabolism in diverse environments,rn00630:Glyoxylate and dicarboxylate metabolism,rn01100:Metabolic pathways,rn01200:Carbon metabolism']
model.reactions.FDH_c.lower_bound = 0.
model.reactions.FDH_c.upper_bound = 1000.
model.reactions.FDH_c.objective_coefficient = 0.

# Confidence Score: Isolated by Zahn et al. 2001
model.reactions.FDH_c.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
model.reactions.FDH_c.notes.update({'LOCALIZATION':['Cytosol']})
# EC number from BRENDA
model.reactions.FDH_c.notes.update({'EC Number':['1.2.1.2']})
# According to 'Respiration in Methanotrophs' Zahn et al. (2005), the enzyme forms a (abcd)2 structure.
# Also functional with (ab)2
# Based on http://www.uniprot.org/uniprot/?query=formate+dehydrogenase+AND+organism%3A%22Methylococcus+capsulatus+%28strain+ATCC+33009+%2F+NCIMB+11132+%2F+Bath%29+%5B243233%5D%22&sort=score
# Gene-Reaction-Rule updated                                alpha                               gamma                    beta               Iron-Sulfur       delta
model.reactions.FDH_c.notes.update({'GENE_ASSOCIATION': ['((MCA2576 or MCA1391 or MCA1210) and (MCA1393 or MCA1208) and (MCA1209 or MCA1392) and MCA1390 and MCA1389)' ]})
model.reactions.FDH_c.gene_reaction_rule = '((MCA2576 or MCA1391 or MCA1210) and (MCA1393 or MCA1208) and (MCA1209 or MCA1392) and MCA1390 and MCA1389)'
# According to 'Respiration in Methanotrophs' Zahn et al. (2005)/ per subunit (abcd)
model.reactions.FDH_c.notes.update({'COFACTOR':['1 FMN or FAD,1 Mo, 5-6 Fe-S clusters']})
# Inhibitor info from same source as above.
model.reactions.FDH_c.notes.update({'INHIBITOR':['']})
model.reactions.FDH_c.notes.update({'BIGG':['FDH']})
model.reactions.FDH_c.notes.update({'MXNREF':['MNXR6148']})
# Check if RXN is mass and charge balanced!
print (model.reactions.FDH_c.check_mass_balance())
# Check RXN-Directionality 
print model.reactions.FDH_c
# Update ID to BiGG
model.repair()

{}
FDH_c: for_c + nad_c --> nadh_c + co2_c


## CYOR_q8 Cytochrome bc1
------------------------

In [36]:
# Fixing ferricytochrome to be Cytochrome cH - Cytochrome c553 - MCA2405
ficytcc553_p = model.metabolites.ficytC_c.copy()
ficytcc553_p.compartment = 'p'
ficytcc553_p.id = 'ficytcc553_p'
ficytcc553_p.formula = 'C42H54FeN8O6S2'
ficytcc553_p.notes['MXNREF'] = ['MNXM11553']
ficytcc553_p.notes['BIGG'] = ['ficytc']
ficytcc553_p.notes['KEGG'] = ['C01070']
ficytcc553_p.notes['CHEBI'] = ['5028','14239','16951']
ficytcc553_p.notes['METACYC'] = ['an oxidized c-type cytochrome']
ficytcc553_p.charge = 3

In [37]:
# Fixing ferrocytochrome to be Cytochrome cH - Cytochrome c553 - MCA2405
# ID confirmed by Blasting N-terminal sequence mentioned in 'Respiration in Methanotrophs' by Zahn et al. 2005
# against Mc genome.
focytcc553_p = model.metabolites.focytC_c.copy()
focytcc553_p.compartment = 'p'
focytcc553_p.id = 'focytcc553_p'
focytcc553_p.formula = 'C42H54FeN8O6S2'
focytcc553_p.notes['MXNREF'] = ['MNXM746']
focytcc553_p.notes['METACYC'] = ['a reduced c-type cytochrome']
focytcc553_p.notes['BIGG'] = ['focytcc553']
focytcc553_p.notes['KEGG'] = ['C01071']
focytcc553_p.notes['CHEBI'] = ['5040','14252','15856']
focytcc553_p.charge = 2

In [38]:
rxn = Reaction('CYOR_q8_im')
rxn.name = 'Ubiquinol-cytochrome c oxidoreductase Complex III'
rxn.subsystem = 'aerobic electron transport chain'
rxn.notes['SUBSYSTEM'] =['aerobic electron transport chain']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({q8h2_im: -1.0,
                          ficytcc553_p: -2.0,
                          focytcc553_p: 2.0,
                          model.metabolites.h_c: -2.0,
                          model.metabolites.h_p: 4.0,
                          model.metabolites.q8_im: 1.0})

In [39]:
# Confidence Score
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.10.2.2']})
# Gene-Reaction-Rule updated
rxn.notes.update({'GENE_ASSOCIATION': ['( MCA1960_c and MCA1961_c and MCA1962 and MCA2405)' ]})
rxn.gene_reaction_rule = '( MCA1960_c and MCA1961_c and MCA1962 and MCA2405)'
# Substrate and cofactor usage from 'Respiration in Methanotrophs' DiSpirito et al. 2005 published in Zannoni's 'Respiration in Archea and Bacteria: Diversity in Prokaryotic Respiration Systems'
rxn.notes.update({'COFACTOR':['Heme B, Heme A, Ubiquinone-8']})
# Inhibitor info from same source as above.
rxn.notes.update({'INHIBITOR':['antimycin a, myxothiazol, stigmatellin']})
rxn.notes.update({'BIGG':['CYOR_q8']})
rxn.notes.update({'MXNREF':['MNXR82078']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'CYOR_q8_im'
model.add_reaction(rxn)

{}
CYOR_q8_im


## Cytochrome c-555 (cL):cytochrome c-553(cH) oxidoreductase
------------------------

In [40]:
# http://www.ebi.ac.uk/interpro/entry/IPR009153
# 'The quinoprotein dehydrogenases for methanol and glucose' Anthony, C. 2004 and Zahn et al. 2005.

rxn = Reaction('CyLCyHOx_p')
rxn.name = 'Cytochrome c-555 (cL):cytochrome c-553(cH) oxidoreductase'
rxn.subsystem = 'rn00680:Methane metabolism,rn01120:Microbial metabolism in diverse environments,rn01100:Metabolic pathways,rn01200:Carbon metabolism'
rxn.notes['SUBSYSTEM'] =['rn00680:Methane metabolism,rn01120:Microbial metabolism in diverse environments,rn01100:Metabolic pathways,rn01200:Carbon metabolism']
rxn.lower_bound = -1000.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({ficytcc555_p: 1.0,
                     focytcc555_p: -1.0,
                     ficytcc553_p: -1.0,
                     focytcc553_p: 1.0})

In [41]:
# Confidence Score
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Periplasm']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.9.-.-']})
# Gene-Reaction-Rule updated
rxn.notes.update({'GENE_ASSOCIATION': ['( MCA2618 and MCA2405)' ]})
rxn.gene_reaction_rule = '( MCA2618 and MCA2405)' 
# Substrate and cofactor usage from 'Respiration in Methanotrophs' DiSpirito et al. 2005 published in Zannoni's 'Respiration in Archea and Bacteria: Diversity in Prokaryotic Respiration Systems'
rxn.notes.update({'COFACTOR':['Heme A']})
# Inhibitor info from same source as above.
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['']})
rxn.notes.update({'MXNREF':['']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'CyLCyHOx_p'
model.add_reaction(rxn)

{}
CyLCyHOx_p


## Terminal Oxidase cytochrome cbd (copper-independent)
-----

In [42]:
rxn = Reaction('CYTCBD_im')
rxn.name = 'cytochrome cbd terminal oxidase - Complex IV'
rxn.subsystem = 'ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways'
rxn.notes['SUBSYSTEM'] =['ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({ficytcc553_p: 4.0,
                     focytcc553_p: -4.0,
                     model.metabolites.o2_c: -1.0,
                     model.metabolites.h_c: -8.0,
                     h_p: 4.0, 
                     model.metabolites.h2o_c: 2.0})

In [43]:
# Confidence Score Suspected presence and function by 
rxn.notes.update({'CONFIDENCE SCORE':['2']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.9.3.1']})
# Gene-Reaction-Rule updated               
rxn.notes.update({'GENE_ASSOCIATION': ['(MCA1185 and MCA1186 and MCA1187)' ]})
rxn.gene_reaction_rule = '(MCA1185 and MCA1186 and MCA1187)' 
# Substrate and cofactor usage from 'Respiration in Methanotrophs' DiSpirito et al. 2005 published in Zannoni's 'Respiration in Archea and Bacteria: Diversity in Prokaryotic Respiration Systems'
rxn.notes.update({'COFACTOR':['2 Heme A, 1 Heme C']})
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['CYOO6m']})
rxn.notes.update({'MXNREF':['MNXR87009']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'CYTCBD_im'
model.add_reaction(rxn)

{}
CYTCBD_im


## Terminal Oxidase cytochrome aa <sub>3</sub>

--------------

In [44]:
rxn = Reaction('CYTAA3_im')
rxn.name = 'cytochrome aa3 terminal oxidase - Complex IV'
rxn.subsystem = 'ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways'
rxn.notes['SUBSYSTEM'] =['ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({ficytcc553_p: 4.0,
                     focytcc553_p: -4.0,
                     model.metabolites.o2_c: -1.0,
                     model.metabolites.h_c: -8.0,
                     h_p: 4.0, 
                     model.metabolites.h2o_c: 2.0})

In [45]:
# Confidence Score Isolated by DiSpirito et al. 'Cytochrome aa3 from Methylococcus capsulatus (Bath)' 1994
rxn.notes.update({'CONFIDENCE SCORE':['4']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.9.3.1']})
# Gene-Reaction-Rule updated                II           I           I & II(possibly ba3)      Assembly     III
rxn.notes.update({'GENE_ASSOCIATION': ['((( MCA0879 and MCA0880) or (MCA2396 and MCA2397)) and MCA0882 and MCA0883)' ]})
rxn.gene_reaction_rule = '((( MCA0879 and MCA0880) or (MCA2396 and MCA2397)) and MCA0882 and MCA0883)' 
# Substrate and cofactor usage from 'Respiration in Methanotrophs' DiSpirito et al. 2005 published in Zannoni's 'Respiration in Archea and Bacteria: Diversity in Prokaryotic Respiration Systems'
rxn.notes.update({'COFACTOR':['2 Heme A, 1 Heme C, 6 Cu2+']})
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['CYOO6m']})
rxn.notes.update({'MXNREF':['MNXR87009']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'CYTAA3_im'
model.add_reaction(rxn)

{}
CYTAA3_im


## ATP Synthase F0F1 - Proton Gradient
--------

In [46]:
rxn = Reaction('ATPSh_im')
rxn.name = 'ATP synthase F0 - H+ - complex V'
rxn.subsystem = 'ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways'
rxn.notes['SUBSYSTEM'] =['ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({model.metabolites.atp_c: 1.0,
                     model.metabolites.adp_c: -1.0,
                     model.metabolites.pi_c: -1.0,
                     model.metabolites.h_c: 3.0,
                     h_p: -4.0, 
                     model.metabolites.h2o_c: 1.0})

In [47]:
# Confidence Score 
rxn.notes.update({'CONFIDENCE SCORE':['2']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['3.6.3.14']})
# Gene-Reaction-Rule updated                A           C          B          delta       alpha       gamma       beta      epsilon         A           C           delta      alpha        gamma         ??       epsilon
rxn.notes.update({'GENE_ASSOCIATION': ['((MCA0006 and MCA0007 and MCA0008 and MCA0009 and MCA0010 and MCA0011 and MCA0012 and MCA0013) or (MCA2699 and MCA2700 and MCA2701 and MCA2707 and MCA2708 and MCA1556 and MCA3021))']})
rxn.gene_reaction_rule = '((MCA0006 and MCA0007 and MCA0008 and MCA0009 and MCA0010 and MCA0011 and MCA0012 and MCA0013) or (MCA2699 and MCA2700 and MCA2701 and MCA2707 and MCA2708 and MCA1556 and MCA3021))' 
# 
rxn.notes.update({'COFACTOR':['']})
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['ATPSh']})
rxn.notes.update({'MXNREF':['MNXR81785']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'ATPSh_im'
model.add_reaction(rxn)

{}
ATPSh_im


## ATP Synthase F0F1 - Na+ Pumping
--------

In [48]:
na1_p = model.metabolites.na1_c.copy()
na1_p.compartment = 'p'
na1_p.id = 'na1_p'
na1_p.formula = 'Na'
na1_p.notes['MXNREF'] = ['MNXM27']
na1_p.notes['BIGG'] = ['na1']
na1_p.notes['KEGG'] = ['C00923']
na1_p.notes['CHEBI'] = ['26710', '29101', '9175', '26717', '49766', '32145', '33167', '63004']
na1_p.notes['METACYC'] = ['NA+']
na1_p.charge = 1

In [49]:
rxn = Reaction('ATPSNa_im')
rxn.name = 'ATP synthase F0 - Na+ - complex V'
rxn.subsystem = 'ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways'
rxn.notes['SUBSYSTEM'] =['ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({model.metabolites.atp_c: 1.0,
                     model.metabolites.adp_c: -1.0,
                     model.metabolites.pi_c: -1.0,
                     model.metabolites.na1_c: 3.0,
                     model.metabolites.h_c: -1.0,
                     na1_p: -3.0, 
                     model.metabolites.h2o_c: 1.0})

In [50]:
# Confidence Score 
rxn.notes.update({'CONFIDENCE SCORE':['2']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['3.6.3.14']})
# Gene-Reaction-Rule updated                A           C          B          delta       alpha       gamma       beta      epsilon         A           C           delta      alpha        gamma         ??         epsilon 
rxn.notes.update({'GENE_ASSOCIATION': ['((MCA0006 and MCA0007 and MCA0008 and MCA0009 and MCA0010 and MCA0011 and MCA0012 and MCA0013) or (MCA2699 and MCA2700 and MCA2701 and MCA2707 and MCA2708 and MCA1556 and MCA3021))']})
rxn.gene_reaction_rule = '((MCA0006 and MCA0007 and MCA0008 and MCA0009 and MCA0010 and MCA0011 and MCA0012 and MCA0013) or (MCA2699 and MCA2700 and MCA2701 and MCA2707 and MCA2708 and MCA1556 and MCA3021))'
# 
rxn.notes.update({'COFACTOR':['']})
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['ATPSh']})
rxn.notes.update({'MXNREF':['MNXR81785']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'ATPSNa_im'
model.add_reaction(rxn)

{}
ATPSNa_im


## Hydrogen-translocating pyrophosphatase
-----

In [51]:
rxn = Reaction('PPApt_im')
rxn.name = 'Proton-translocating pyrophosphatase'
rxn.subsystem = 'ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways'
rxn.notes['SUBSYSTEM'] =['ec00190:Oxidative phosphorylation,ec01100:Metabolic pathways']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({model.metabolites.ppi_c: -1.0,
                     model.metabolites.h2o_c: -1.0,
                     model.metabolites.pi_c: 2.0,
                     model.metabolites.h_c: -1.0,
                     h_p: 2.0})

In [52]:
# Confidence Score 
rxn.notes.update({'CONFIDENCE SCORE':['2']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['3.6.1.1']})
# Gene-Reaction-Rule updated               
rxn.notes.update({'GENE_ASSOCIATION': ['(MCA1188)']})
rxn.gene_reaction_rule = '(MCA1188)' 
# 
rxn.notes.update({'COFACTOR':['']})
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['PPA']})
rxn.notes.update({'MXNREF':['MNXR4373']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'PPApt_im'
model.add_reaction(rxn)

{}
PPApt_im


## NDH-1
-------

In [53]:
rxn = Reaction('NADH16_im')
rxn.name = 'type-1 nadh:quinone oxidoreductase (H+-translocating)'
rxn.subsystem = 'ec00190:Oxidative phosphorylation, ec01100:Metabolic pathways'
rxn.notes['SUBSYSTEM'] =['ec00190:Oxidative phosphorylation, ec01100:Metabolic pathways']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({q8h2_im: 1.0,
                          model.metabolites.nad_c: 1.0,
                          model.metabolites.nadh_c: -1.0,
                          model.metabolites.h_c: -4.0,
                          model.metabolites.q8_im: -1.0,
                          h_p: 3.0})

In [54]:
# Confidence Score
rxn.notes.update({'CONFIDENCE SCORE':['2']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.6.5.11']})
# Gene-Reaction-Rule updated             N           M           L           K           J           I           H           G           F           E           C/D         B           A
rxn.notes.update({'GENE_ASSOCIATION': ['(MCA1347 and MCA1348 and MCA1349 and MCA1350 and MCA1351 and MCA1352 and MCA1353 and MCA1354 and MCA1355 and MCA1356 and MCA1357 and MCA1358 and MCA1359)' ]})
rxn.gene_reaction_rule = '(MCA1347 and MCA1348 and MCA1349 and MCA1350 and MCA1351 and MCA1352 and MCA1353 and MCA1354 and MCA1355 and MCA1356 and MCA1357 and MCA1358 and MCA1359)'
# Substrate and cofactor usage from BRENDA
# Uses only NADH
rxn.notes.update({'COFACTOR':['FMN,Fe-S clusters']})
# Inhibitor info from Cook & Shiemke et al. 2004 doi: 10.1128/JB.186.4.928-937.2004
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['NADH16']})
rxn.notes.update({'MXNREF':['']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'NADH16_im'
model.add_reaction(rxn)

{}
NADH16_im


## NQR
-------

In [55]:
rxn = Reaction('NQR_im')
rxn.name = 'Na(+)-translocating NADH-quinone - NQR'
rxn.subsystem = 'ec00190:Oxidative phosphorylation, ec01100:Metabolic pathways'
rxn.notes['SUBSYSTEM'] =['ec00190:Oxidative phosphorylation, ec01100:Metabolic pathways']
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.objective_coefficient = 0.
rxn.add_metabolites({q8h2_im: 1.0,
                          model.metabolites.nad_c: 1.0,
                          model.metabolites.nadh_c: -1.0,
                          model.metabolites.h_c: -1.0,
                          model.metabolites.q8_im: -1.0,
                          model.metabolites.na1_c: 2.0,
                          na1_p: -2.0, })

In [56]:
# Confidence Score
rxn.notes.update({'CONFIDENCE SCORE':['2']})
# Localization
rxn.notes.update({'LOCALIZATION':['Inner Membrane']})
# EC number from BRENDA
rxn.notes.update({'EC Number':['1.6.5.8']})
# Gene-Reaction-Rule updated             A           B           C           D           E           F 
rxn.notes.update({'GENE_ASSOCIATION': ['(MCA2389 and MCA2388 and MCA2387 and MCA2386 and MCA2385 and MCA2384)']})
rxn.gene_reaction_rule = '(MCA2389 and MCA2388 and MCA2387 and MCA2386 and MCA2385 and MCA2384)'
# Substrate and cofactor usage from BRENDA
# Uses only NADH
rxn.notes.update({'COFACTOR':['FMN,FAD,Fe-S clusters']})
rxn.notes.update({'INHIBITOR':['']})
rxn.notes.update({'BIGG':['NADHNQR']})
rxn.notes.update({'MXNREF':['']})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Check RXN-Directionality 
print rxn
# Update ID to BiGG
rxn.id = 'NQR_im'
model.add_reaction(rxn)

{}
NQR_im


## Clean-up of unused respiratory reactions and metabolites

In [57]:
model.remove_reactions([model.reactions.ALCD1_c], delete=True, remove_orphans=True)

In [58]:
model.remove_reactions([model.reactions.MNXR10335_c], delete=True, remove_orphans=True)
model.remove_reactions([model.reactions.MNXR84835_c], delete=True, remove_orphans=True)
model.remove_reactions([model.reactions.PRDX_c], delete=True, remove_orphans=True)
model.remove_reactions([model.reactions.MNXR6148_c], delete=True, remove_orphans=True)
model.remove_reactions([model.reactions.FHL_c], delete=True, remove_orphans=True)
model.remove_reactions([model.reactions.MNXR84917_c], delete=True, remove_orphans=True)

In [59]:
target_filename = relative_directory + '/Reconstructions/MethylococcusModel3.xml'
cobra.io.write_legacy_sbml(model, target_filename,use_fbc_package=False)

## Map in Escher

In [60]:
# import escher
# import csv
# import math

# reader = csv.reader(open('/Users/clie/Desktop/EFPro2/Experimental Data & Standards/Transcriptomics/RNAseq/Transcriptomic Data Cu-Difference Escher copy.csv',mode='rU'))
# # mydict = {rows[0]:[rows[1], rows[2]] for rows in reader}

# mydict = {}
# for rows in reader:
#     try: 
#         fold_change = float(rows[1])/float(rows[2])
#         log_fold_change = math.log(fold_change,2)
#         mydict.update({rows[0]:[log_fold_change]})
#     except:
#         mydict.update({rows[0]:0.0})
        
# maps = escher.Builder(
# #     model=model,
#                     map_json='/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/Respiratory_Chain.json',
#                     gene_data=mydict,
#                     show_gene_reaction_rules=True,
#                     use_3d_transform = True
#                     )

# maps.display_in_browser()